In [1]:
import numpy as np
import pandas as pd
import math
import requests
import xlsxwriter
from scipy import stats

In [2]:
stonks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN


In [3]:
symbol = 'AAPL'
api_call = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_call).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2238790870228,
 'week52high': 151.48,
 'week52low': 78.2,
 'week52highSplitAdjustOnly': 149.6,
 'week52lowSplitAdjustOnly': 76.28,
 'week52change': 0.7256954560583112,
 'sharesOutstanding': 17086307088,
 'float': 0,
 'avg10Volume': 108511587,
 'avg30Volume': 95600642,
 'day200MovingAvg': 129.52,
 'day50MovingAvg': 131.44,
 'employees': 150993,
 'ttmEPS': 4.65,
 'ttmDividendRate': 0.8394728727700105,
 'dividendYield': 0.006358692826918357,
 'nextDividendDate': '0',
 'exDividendDate': '2021-04-24',
 'nextEarningsDate': '2021-07-17',
 'peRatio': 29.31386181746087,
 'beta': 1.3764781312133398,
 'maxChangePercent': 50.733391858055,
 'year5ChangePercent': 5.00955915988168,
 'year2ChangePercent': 1.58997407263079,
 'year1ChangePercent': 0.7474774093174235,
 'ytdChangePercent': -0.020999485917386016,
 'month6ChangePercent': 0.09605596621349206,
 'month3ChangePercent': -0.05362753589954675,
 'month1ChangePercent': 0.028906017720545345,
 'day30ChangePer

In [4]:
def divide_chunks(l, n):
      
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]
        
ticker_group = list(divide_chunks(stonks["Ticker"],100))
symbol_strings = []
for i in range(0,len(ticker_group)):
    symbol_strings.append(','.join(ticker_group[i]))


In [5]:
my_columns = ['Ticker','price', 'one year price return','stocks to buy']
output_dataframe = pd.DataFrame(columns = my_columns)
output_dataframe

,Ticker,price,one year price return,stocks to buy


In [6]:
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    info = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        output_dataframe = output_dataframe.append(
        pd.Series(
            [symbol,
             info[symbol]['price'],
             info[symbol]['stats']['year1ChangePercent'],
             'NA'  
            ],
            index = my_columns                                 
        ),
        ignore_index = True
        )
output_dataframe    
        

,Ticker,price,one year price return,stocks to buy
0,A,137.70,0.723825,NA
1,AAL,22.41,1.36881,NA
2,AAP,210.02,0.712336,NA
3,AAPL,134.36,0.773536,NA
4,ABBV,118.72,0.446488,NA
...,...,...,...,...
500,YUM,125.86,0.501558,NA
501,ZBH,177.79,0.52718,NA
502,ZBRA,474.56,1.06948,NA
503,ZION,61.53,1.07187,NA


In [7]:
output_dataframe.sort_values('one year price return',ascending=False,inplace=True)
output_dataframe = output_dataframe[:50]
output_dataframe.reset_index(inplace = True)
output_dataframe

,index,Ticker,price,one year price return,stocks to buy
0,275,LB,67.31,5,NA
1,179,FCX,42.30,4.03089,NA
2,208,GPS,35.38,3.90413,NA
3,253,IVZ,28.77,3.03968,NA
4,272,KSS,60.74,2.64429,NA
5,441,TPR,47.20,2.40534,NA
6,314,MOS,36.67,2.31995,NA
7,410,SIVB,595.59,2.29925,NA
8,462,URI,358.39,2.15005,NA
9,129,DFS,118.61,2.13717,NA


In [8]:

def portfolio_input() : 
    global portfolio_size
    portfolio_size = input('enter your portfolio size in rupees')
    try :
        float(portfolio_size)
    except ValueError:
        portfolio_size = input('pls enter the value in integers')
    val = float(portfolio_size)
    
# StoksToBuy*Price = x(i)  sumx(i)=portfoliosize
# x(i) = portfolioSize*1yearreturn/sum(1yearreturn)
# total_return = output_dataframe['one year price return'].sum()
# for i in range(50) :
#     val = (float(portfolio_size)*output_dataframe.loc[i,'one year price return'])/total_return
#     output_dataframe.loc[i,'stocks to buy'] = math.floor(val/output_dataframe.loc[i,'price'])
    
# output_dataframe
portfolio_input()
# output_dataframe
weight = float(portfolio_size)/len(output_dataframe.index)
for i in range(0, len(output_dataframe)):
    output_dataframe.loc[i,'stocks to buy'] = math.floor(weight/output_dataframe.loc[i,'price'])
output_dataframe


enter your portfolio size in rupees1000000


C:\Users\BB\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,price,one year price return,stocks to buy
0,275,LB,67.31,5,297
1,179,FCX,42.30,4.03089,472
2,208,GPS,35.38,3.90413,565
3,253,IVZ,28.77,3.03968,695
4,272,KSS,60.74,2.64429,329
5,441,TPR,47.20,2.40534,423
6,314,MOS,36.67,2.31995,545
7,410,SIVB,595.59,2.29925,33
8,462,URI,358.39,2.15005,55
9,129,DFS,118.61,2.13717,168


high Quality momentum strategy

1 month 
3month
6 month
1 year price return

In [56]:
hqm_columns = [
               'ticker',
               'price',
               '1 month return',
               '1 month return percentile',
               '3 month return',
               '3 month return percentile',
               '6 month return',
               '6 month return percentile',
               '1 year return',
               '1 year return percentile',
               'HQM score',
               'stocks to buy'               
              ]
hqm_final_dataframe = pd.DataFrame(columns = hqm_columns)


for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()   
    
    for symbol in symbol_string.split(','):
        
        hqm_final_dataframe=hqm_final_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['month1ChangePercent'],
            'Nan',
            data[symbol]['stats']['month3ChangePercent'],
            'Nan',
            data[symbol]['stats']['month6ChangePercent'],
            'Nan',
            data[symbol]['stats']['year1ChangePercent'],
            'Nan',
            'Nan',
            'Nan'
            ],
            index = hqm_columns
        ),
                 
        ignore_index = True
        
        )
        
hqm_final_dataframe    

,ticker,price,1 month return,1 month return percentile,3 month return,3 month return percentile,6 month return,6 month return percentile,1 year return,1 year return percentile,HQM score,stocks to buy
0,A,138.14,0.0107695,Nan,0.0782108,Nan,0.182315,Nan,0.723741,Nan,Nan,Nan
1,AAL,22.20,-0.109131,Nan,0.250304,Nan,0.911541,Nan,1.3284,Nan,Nan,Nan
2,AAP,213.36,0.11432,Nan,0.303519,Nan,0.312722,Nan,0.730031,Nan,Nan,Nan
3,AAPL,131.23,0.0292948,Nan,-0.05308,Nan,0.0968594,Nan,0.773724,Nan,Nan,Nan
4,ABBV,116.72,0.115933,Nan,0.0817455,Nan,0.281976,Nan,0.44442,Nan,Nan,Nan
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.54,0.0651682,Nan,0.160538,Nan,0.238017,Nan,0.489325,Nan,Nan,Nan
501,ZBH,176.37,0.0598805,Nan,0.0867585,Nan,0.261349,Nan,0.521868,Nan,Nan,Nan
502,ZBRA,489.65,-0.0413437,Nan,0.162937,Nan,0.369146,Nan,1.04302,Nan,Nan,Nan
503,ZION,60.67,0.0698581,Nan,0.250041,Nan,0.906741,Nan,1.07125,Nan,Nan,Nan


calculating momentum percentiles


In [57]:
time_periods = ['1 month',
               '3 month',
               '6 month',
               '1 year',
               ]

for row in hqm_final_dataframe.index:
    for time_period in time_periods:
        return_col = f'{time_period} return'
        percentile_col = f'{time_period} return percentile'
        if hqm_final_dataframe.loc[row, return_col] == None:
            hqm_final_dataframe.loc[row, return_col] = 0.0
            
for row in hqm_final_dataframe.index:
    for time_period in time_periods:
        return_col = f'{time_period} return'
        percentile_col = f'{time_period} return percentile'
        hqm_final_dataframe.loc[row,percentile_col] = stats.percentileofscore(hqm_final_dataframe[return_col],hqm_final_dataframe.loc[row,return_col])

hqm_final_dataframe

,ticker,price,1 month return,1 month return percentile,3 month return,3 month return percentile,6 month return,6 month return percentile,1 year return,1 year return percentile,HQM score,stocks to buy
0,A,138.14,0.0107695,28.7129,0.0782108,29.3069,0.182315,35.0495,0.723741,60.5941,Nan,Nan
1,AAL,22.20,-0.109131,3.16832,0.250304,78.2178,0.911541,93.6634,1.3284,88.5149,Nan,Nan
2,AAP,213.36,0.11432,86.5347,0.303519,90.495,0.312722,53.8614,0.730031,60.7921,Nan,Nan
3,AAPL,131.23,0.0292948,38.2178,-0.05308,7.52475,0.0968594,22.5743,0.773724,63.3663,Nan,Nan
4,ABBV,116.72,0.115933,87.1287,0.0817455,30.099,0.281976,49.901,0.44442,36.6337,Nan,Nan
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.54,0.0651682,63.3663,0.160538,54.0594,0.238017,42.3762,0.489325,41.7822,Nan,Nan
501,ZBH,176.37,0.0598805,58.0198,0.0867585,31.8812,0.261349,46.1386,0.521868,43.7624,Nan,Nan
502,ZBRA,489.65,-0.0413437,13.0693,0.162937,55.0495,0.369146,60.9901,1.04302,77.2277,Nan,Nan
503,ZION,60.67,0.0698581,66.7327,0.250041,78.0198,0.906741,93.2673,1.07125,78.6139,Nan,Nan


In [58]:
from statistics import mean

for row in hqm_final_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_final_dataframe.loc[row, f'{time_period} return percentile'])
    hqm_final_dataframe.loc[row,'HQM score'] = mean(momentum_percentiles)





sort hqm scores in decreasing order

In [59]:
hqm_final_dataframe.sort_values('HQM score', ascending = False, inplace = True)
hqm_final_dataframe = hqm_final_dataframe[:50]
hqm_final_dataframe.reset_index(inplace = True, drop = True)

hqm_final_dataframe


,ticker,price,1 month return,1 month return percentile,3 month return,3 month return percentile,6 month return,6 month return percentile,1 year return,1 year return percentile,HQM score,stocks to buy
0,FCX,44.220,0.207366,98.6139,0.337355,94.0594,1.21494,98.2178,3.9815,99.802,97.6733,Nan
1,COF,157.470,0.190235,97.8218,0.369658,96.6337,1.02626,95.8416,1.75452,96.2376,96.6337,Nan
2,NUE,98.400,0.207207,98.4158,0.837855,100,1.04463,96.0396,1.44241,91.0891,96.3861,Nan
3,MOS,37.660,0.147045,95.6436,0.297615,88.5149,1.1428,97.6238,2.42884,99.0099,95.198,Nan
4,MHK,232.360,0.125105,89.901,0.492317,99.604,0.880383,92.0792,1.89363,97.6238,94.802,Nan
5,TXT,67.210,0.184597,97.6238,0.409851,97.6238,0.768399,87.5248,1.75207,96.0396,94.703,Nan
6,GPS,35.537,0.130638,92.2772,0.572523,99.802,0.659649,82.3762,3.8962,99.604,93.5149,Nan
7,DVN,26.230,0.111019,85.7426,0.441768,98.6139,1.66764,99.2079,1.39028,90.297,93.4653,Nan
8,DFS,120.030,0.194549,98.0198,0.277337,86.5347,0.778255,88.5149,2.16744,98.4158,92.8713,Nan
9,LNC,69.560,0.0973557,80.9901,0.36595,96.4356,1.06741,96.6337,1.40631,90.6931,91.1881,Nan


In [60]:
portfolio_input()

enter your portfolio size in rupees10000000


In [62]:
position_size = float(portfolio_size)/len(hqm_final_dataframe.index)

for row in hqm_final_dataframe.index:
    hqm_final_dataframe.loc[row, 'stocks to buy'] = math.floor(position_size/hqm_final_dataframe.loc[row, 'price'])
    
hqm_final_dataframe

C:\Users\BB\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,ticker,price,1 month return,1 month return percentile,3 month return,3 month return percentile,6 month return,6 month return percentile,1 year return,1 year return percentile,HQM score,stocks to buy
0,FCX,44.220,0.207366,98.6139,0.337355,94.0594,1.21494,98.2178,3.9815,99.802,97.6733,4522
1,COF,157.470,0.190235,97.8218,0.369658,96.6337,1.02626,95.8416,1.75452,96.2376,96.6337,1270
2,NUE,98.400,0.207207,98.4158,0.837855,100,1.04463,96.0396,1.44241,91.0891,96.3861,2032
3,MOS,37.660,0.147045,95.6436,0.297615,88.5149,1.1428,97.6238,2.42884,99.0099,95.198,5310
4,MHK,232.360,0.125105,89.901,0.492317,99.604,0.880383,92.0792,1.89363,97.6238,94.802,860
5,TXT,67.210,0.184597,97.6238,0.409851,97.6238,0.768399,87.5248,1.75207,96.0396,94.703,2975
6,GPS,35.537,0.130638,92.2772,0.572523,99.802,0.659649,82.3762,3.8962,99.604,93.5149,5627
7,DVN,26.230,0.111019,85.7426,0.441768,98.6139,1.66764,99.2079,1.39028,90.297,93.4653,7624
8,DFS,120.030,0.194549,98.0198,0.277337,86.5347,0.778255,88.5149,2.16744,98.4158,92.8713,1666
9,LNC,69.560,0.0973557,80.9901,0.36595,96.4356,1.06741,96.6337,1.40631,90.6931,91.1881,2875
